# Import modules


In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#ML auxiliary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn

#SkLearn auxiliary libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, ConfusionMatrixDisplay, f1_score, precision_score, recall_score, top_k_accuracy_score

#SkLearn classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd 
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense, Dropout
import keras.backend as K
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix
from keras.utils.np_utils import to_categorical
from keras.metrics import TopKCategoricalAccuracy

In [3]:
!pip install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.3 MB/s 


# Import datasets

## Our dataset:
- API calls: **48 API**
- Families: 12 ransomware + 1 benign -> **5203 Samples**

In [8]:
our_dataset = pd.read_csv('/content/drive/MyDrive/dataset_final.csv')

api_count = our_dataset.drop('Family', axis=1).sum()
print("\n\nAPIs:")
print(api_count)

data = our_dataset.to_numpy()
x_ours = data[:, :-1].astype(float)
y_ours = data[:, -1].astype(float)
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_ours, test_size=0.2)

print("\n\nFamilies:")
family_count = our_dataset['Family']
print(family_count.value_counts())



APIs:
CreateProcessInternalW      2020
CreateServiceA               143
CreateServiceW               786
CryptExportKey               145
CryptGenKey                  134
CryptHashData                 41
DeviceIoControl               88
EnumServicesStatusA           83
EnumServicesStatusW           34
FindWindowA                  152
GetAdaptersAddresses        1499
GetComputerNameA             649
GetComputerNameW            3227
GetDiskFreeSpaceExW         1399
GetDiskFreeSpaceW            195
GlobalMemoryStatusEx        3527
InternetOpenA                308
IsDebuggerPresent           3631
LdrGetDllHandle               31
LookupPrivilegeValueW       2747
MoveFileWithProgressW        197
NtAllocateVirtualMemory     6215
NtCreateFile                 138
NtCreateKey                  737
NtGetContextThread           627
NtLoadDriver                  21
NtMapViewOfSection           106
NtProtectVirtualMemory      6224
NtQuerySystemInformation      46
NtResumeThread               617
Nt

## Paranoia Dataset

In [ ]:
paranoia_dataset = pd.read_csv('/content/drive/MyDrive/paranoia_dataset.csv')
data_para = paranoia_dataset.to_numpy()
x_para = data_para[:, :-1].astype(float)
y_para = data_para[:, -1].astype(float)

# Our model with Paranoia Dataset

* Accuracy = **93.24%**
* Top-k Categorical Accuracy = **98.77%**
* Recall = **91.00%**
* Precision = **91.00%**
* F1 Score = **91.00%**

In [ ]:
y_cat_para = to_categorical(y_para)
x_train_para, x_test_para, y_train_para, y_test_para = train_test_split(x_para, y_cat_para, test_size=0.2)

model = Sequential()
model.add(Dense(512, input_dim=23, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(6, activation = "softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', TopKCategoricalAccuracy(k=2)])
model.summary()
model.fit(x_train_para, y_train_para, verbose=1, epochs=100, batch_size=15)

predict_x_para=model.predict(x_test_para) 
y_pred_class_para=np.argmax(predict_x_para,axis=1)

y_pred_para = model.predict(x_test_para)
y_test_class_para = np.argmax(y_test_para, axis=1)
print(confusion_matrix(y_test_class_para, y_pred_class_para))

print(classification_report(y_test_class_para, y_pred_class_para))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               12288     
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                                 
Total params: 177,286
Trainable params: 177,286
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
271/271 [=============================

# Our model with our dataset

* Accuracy = **82.29%**
* Top-k Categorical Accuracy = **90.51%**
* Recall = **80.00%**
* Precision = **78.00%**
* F1 Score = **78.00%**

In [ ]:
y_cat_ours = to_categorical(y_ours)
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x_ours, y_cat_ours, test_size=0.2)

model = Sequential()
model.add(Dense(512, input_dim=48, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(24, activation = "softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', TopKCategoricalAccuracy(k=2)])
model.summary()
model.fit(x_train_ours, y_train_ours, verbose=1, epochs=100, batch_size=10)

predict_x=model.predict(x_test_ours) 
y_pred_class=np.argmax(predict_x,axis=1)

y_pred = model.predict(x_test_ours)
y_test_class = np.argmax(y_test_ours, axis=1)
print(confusion_matrix(y_test_class, y_pred_class))

print(classification_report(y_test_class, y_pred_class))

# Paranoia Model with Paranoia Dataset

* Accuracy = **89.04%**
* Top-k Categorical Accuracy = **-**
* Recall = **89.04%**
* Precision = **92.82%**
* F1 Score = **90.84%**

In [ ]:
x_train_para, x_test_para, y_train_para, y_test_para = train_test_split(x_para, y_para, test_size=0.2)

randclf = RandomForestClassifier(n_estimators=50, random_state=10)
randclf.fit(x_train_para, y_train_para)
y_train_pred_para = randclf.predict(x_train_para)
print('train precision: ' + str(precision_score(y_train_para, y_train_pred_para, average='weighted')))
print('train recall: ' + str(recall_score(y_train_para, y_train_pred_para, average='weighted')))
print('train f1-score: ' + str(f1_score(y_train_para, y_train_pred_para, average='weighted')))
# print('train top_k_accuracy_score: ' + str(top_k_accuracy_score(y_train, y_train_pred, k=2)))
print('train accuracy: ' + str(accuracy_score(y_train_para, y_train_pred_para)))

y_test_pred_para = randclf.predict(x_test_para)
print('test precision: ' + str(precision_score(y_test_para, y_test_pred_para, average='weighted')))
print('test recall: ' + str(recall_score(y_test_para, y_test_pred_para, average='weighted')))
print('test f1-score: ' + str(f1_score(y_test_para, y_test_pred_para, average='weighted')))
print('test top_k_accuracy_score: ' + str(top_k_accuracy_score(y_test_para, randclf.predict_proba(x_test_para), k=2)))
print('test accuracy: ' + str(accuracy_score(y_test_para, y_test_pred_para)))

train precision: 0.9347182094400233
train recall: 0.9333497901752653
train f1-score: 0.9330554016938052
train accuracy: 0.9333497901752653
test precision: 0.9217313229125541
test recall: 0.9200394866732478
test f1-score: 0.9194539470766399
test top_k_accuracy_score: 0.9753208292201382
test accuracy: 0.9200394866732478


# Paranoia Model with Our Dataset

* Accuracy = **71.37%**
* Top-k Categorical Accuracy = **-**
* Recall = **71.37%**
* Precision = **90.19%**
* F1 Score = **78.88%**

In [ ]:
randclf = RandomForestClassifier(n_estimators=50, random_state=10)
randclf.fit(x_train_ours, y_train_ours)
y_train_pred_ours = randclf.predict(x_train_ours)
print('train precision: ' + str(precision_score(y_train_ours, y_train_pred_ours, average='weighted')))
print('train recall: ' + str(recall_score(y_train_ours, y_train_pred_ours, average='weighted')))
print('train f1-score: ' + str(f1_score(y_train_ours, y_train_pred_ours, average='weighted')))
print('train accuracy: ' + str(accuracy_score(y_train_ours, y_train_pred_ours)))

y_test_pred_ours = randclf.predict(x_test_ours)
print('test precision: ' + str(precision_score(y_test_ours, y_test_pred_ours, average='weighted')))
print('test recall: ' + str(recall_score(y_test_ours, y_test_pred_ours, average='weighted')))
print('test f1-score: ' + str(f1_score(y_test_ours, y_test_pred_ours, average='weighted')))
print('test accuracy: ' + str(accuracy_score(y_test_ours, y_test_pred_ours)))
print('test top_k_accuracy_score: ' + str(top_k_accuracy_score(y_test_ours, randclf.predict_proba(x_test_ours), k=2)))

train precision: 0.8378736449973225
train recall: 0.826045170591062
train f1-score: 0.8279886509422316
train accuracy: 0.826045170591062
test precision: 0.7929147294167281
test recall: 0.7877041306436119
test f1-score: 0.7878844031096703
test accuracy: 0.7877041306436119
test top_k_accuracy_score: 0.8674351585014409


# Other models

## Bernoulli Naive Bayes

In [ ]:
bern_na_bay = BernoulliNB()

scores = cross_val_score(bern_na_bay, x_ours, y_ours, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x_ours, y_ours, test_size=0.2, stratify=y, random_state=75)
bern_na_bay.fit(x_train, y_train)

predict=bern_na_bay.predict(x_test)

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))
print('top-k accuracy: ' + str(top_k_accuracy_score(y_test, bern_na_bay.predict_proba(x_test), k=2)))

0.5631363502140853
precision: 0.6206699660901371
recall: 0.5658021133525456
f1-score: 0.567376232014625
accuracy: 0.5658021133525456
accuracy: 0.723342939481268


## Random Forest

In [ ]:
rand_for_classifier=RandomForestClassifier()

scores = cross_val_score(rand_for_classifier, x_ours, y_ours, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x_ours, y_ours, test_size=0.2, random_state=75)
rand_for_classifier.fit(x_train, y_train)

predict=rand_for_classifier.predict(x_test) 

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))
print('top-k accuracy: ' + str(top_k_accuracy_score(y_test, rand_for_classifier.predict_proba(x_test), k=2)))

0.7758980510851912
precision: 0.8144251543440643
recall: 0.7905859750240154
f1-score: 0.7885681018392904
accuracy: 0.7905859750240154
top-k accuracy: 0.8587896253602305


## K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

scores = cross_val_score(knn, x_ours, y_ours, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x_ours, y_ours, test_size=0.2, random_state=75)
knn.fit(x_train, y_train)

predict=knn.predict(x_test) 

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))
print('top-k accuracy: ' + str(top_k_accuracy_score(y_test, knn.predict_proba(x_test), k=2)))

0.6515439982282591
precision: 0.7725907999734125
recall: 0.7521613832853026
f1-score: 0.7534116062348658
accuracy: 0.7521613832853026
top-k accuracy: 0.8203650336215178


# Binary classification

### Dataset preparation
First of all we transform the dataset to a two-class one by replacing all families identifiers with *Ransomware* or *Benign* classes.

In [ ]:
binary_class_dataset = our_dataset.copy(deep=True)

In [ ]:
binary_class_dataset['Family'] = np.where(binary_class_dataset.Family <= 22, 'Ransomware', 'Benign')

In [ ]:
binary_class_dataset['Family'] = np.where(binary_class_dataset.Family <= 22, 1, 0)

In [ ]:
binary_class_dataset.shape

(5203, 49)

We then split the labels from the features in order to run the classification tasks.

In [ ]:
data = binary_class_dataset.to_numpy()
x = data[:, :-1].astype(int)
y = data[:, -1].astype(int)

### ML Models
In this section we train and validate the three models which have been already mentioned above for what concerns binary classification

#### Bernoulli Naive Bayes

In [ ]:
bern_na_bay = BernoulliNB()

scores = cross_val_score(bern_na_bay, x, y, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=75)
bern_na_bay.fit(x_train, y_train)

predict=bern_na_bay.predict(x_test) 

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))

0.9978857227225749
precision: 0.9981160758419368
recall: 0.9980787704130644
f1-score: 0.998087165186208
accuracy: 0.9980787704130644


#### K-Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

scores = cross_val_score(knn, x, y, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=75)
knn.fit(x_train, y_train)

predict=knn.predict(x_test) 

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))

0.9969241842610364
precision: 0.9980828668514158
recall: 0.9980787704130644
f1-score: 0.9980705589349123
accuracy: 0.9980787704130644


#### Random Forest

In [ ]:
rand_for_classifier=RandomForestClassifier()

scores = cross_val_score(rand_for_classifier, x, y, cv=10)
print(scores.mean())

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=75)
rand_for_classifier.fit(x_train, y_train)

predict=rand_for_classifier.predict(x_test) 

print('precision: ' + str(precision_score(predict, y_test, average='weighted')))
print('recall: ' + str(recall_score(predict, y_test, average='weighted')))
print('f1-score: ' + str(f1_score(predict, y_test, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict, y_test)))

0.9992311383434224
precision: 1.0
recall: 1.0
f1-score: 1.0
accuracy: 1.0


## Neural Network

In [ ]:
x_train_ours, x_test_ours, y_train_ours, y_test_ours = train_test_split(x, y, test_size=0.2)

model = Sequential()
model.add(Dense(512, input_dim=48, activation = "relu"))
model.add(Dense(256, activation = "relu"))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation = "softmax"))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', TopKCategoricalAccuracy(k=2)])
model.summary()
model.fit(x_train_ours, y_train_ours, verbose=1, epochs=100, batch_size=10)

predict_x=model.predict(x_test_ours)

print('precision: ' + str(precision_score(predict_x, y_test_ours, average='weighted')))
print('recall: ' + str(recall_score(predict_x, y_test_ours, average='weighted')))
print('f1-score: ' + str(f1_score(predict_x, y_test_ours, average='weighted')))
print('accuracy: ' + str(accuracy_score(predict_x, y_test_ours)))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 512)               25088     
                                                                 
 dense_17 (Dense)            (None, 256)               131328    
                                                                 
 dense_18 (Dense)            (None, 128)               32896     
                                                                 
 dropout_4 (Dropout)         (None, 128)               0         
                                                                 
 dense_19 (Dense)            (None, 1)                 129       
                                                                 
Total params: 189,441
Trainable params: 189,441
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
417/417 [===========================

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
